In [0]:
# !pip install fastai==0.7.0 <- will need to run this the first time to get the FASTAI library imported

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
from sklearn.preprocessing import MinMaxScaler
from google.colab import files
from fastai.structured import  add_datepart
from sklearn.linear_model import LinearRegression
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [0]:
# Import CSV File and Store in DataFrame - Take Data From Quandl
uploaded = files.upload()
print(str(uploaded))
filename = uploaded['EOD-KO (1).csv']
import io
df = pd.read_csv(io.BytesIO(filename))


In [0]:
# Set Date To Be Index
df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
df.index = df['Date']

# Plotting Details
plt.figure(figsize=(16,8))
plt.plot(df['Close'], label='Close Price history')

In [0]:
# Set Date To Be Index
df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
df.index = df['Date']

# Sort Data In Increasing Order 
data = df.sort_index(ascending=True, axis=0)

# Create a Copy Of The Dataset
copiedData = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])

for i in range(0,len(data)):
    copiedData['Date'][i],copiedData['Close'][i] = data['Date'][i],data['Close'][i]

# Decide on Features
add_datepart(copiedData, 'Date')
copiedData.drop('Elapsed', axis=1, inplace=True)  #elapsed will be the time stamp

copiedData['mon_fri'] = 0
for i in range(0,len(copiedData)):
    copiedData['mon_fri'][i] = 1 if (copiedData['Dayofweek'][i] == 0 or copiedData['Dayofweek'][i] == 4) else 0

# Test and Test Sets
train,test  = copiedData[:1000],copiedData[1000:]

xTrain,yTrain  = train.drop('Close', axis=1),train['Close']
xTest,yTest = test.drop('Close', axis=1),test['Close']

# Linear Regression Model 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xTrain,yTrain)

# Predict Values and Calculate Errors
predictedValues = model.predict(xTest)
rootMeanSquaredError=np.sqrt(np.mean(np.power((np.array(yTest)-np.array(predictedValues)),2)))
rootMeanSquaredError

# Plot Results
test['Predictions'] = 0
test['Predictions'] = predictedValues

test.index,train.index = copiedData[1000:].index,copiedData[:1000].index

plt.plot(train['Close'])
plt.plot(test[['Close', 'Predictions']])
plt.title("LINEAR REGRESSION MODEL")

In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale Data
xTrain_scaled = scaler.fit_transform(xTrain)
xTrain = pd.DataFrame(xTrain_scaled)
xTest_scaled = scaler.fit_transform(xTest)
xTest = pd.DataFrame(xTest_scaled)

# Find Best Parameters For K-NN
params = {'n_neighbors':[2,3,4,5,6,7,8,9]}
knn = neighbors.KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)

# Predicted Values and rootMeanSquaredErrorE Calculation
model.fit(xTrain,yTrain)
predictedValues = model.predict(xTest)
rootMeanSquaredError=np.sqrt(np.mean(np.power((np.array(yTest)-np.array(predictedValues)),2)))

# Plot Graph
test['Predictions'] = 0
test['Predictions'] = predictedValues
plt.plot(train['Close'])
plt.plot(test[['Close', 'Predictions']])
plt.title("KNN MODEL")

In [0]:
# Store Data in DataFrame Object
data = df.sort_index(ascending=True, axis=0)
copiedData = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    copiedData['Date'][i],copiedData['Close'][i] = data['Date'][i],data['Close'][i]

# Change The Index To Use Date
copiedData.index = copiedData.Date
copiedData.drop('Date', axis=1, inplace=True)

# Create Train and Test Sets
dataset = copiedData.values

train,test = dataset[0:1000,:],dataset[1000:,:]

# Scale The Test Sets
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

xTrain, yTrain = [], []
for i in range(60,len(train)):
    xTrain.append(scaled_data[i-60:i,0])
    yTrain.append(scaled_data[i,0])
    
xTrain, yTrain = np.array(xTrain), np.array(yTrain)
xTrain = np.reshape(xTrain, (xTrain.shape[0],xTrain.shape[1],1))

# Run LSTM Model on Fitted Data
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(xTrain.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(xTrain, yTrain, epochs=1, batch_size=1, verbose=2)

# Use Last 60 Days of Data to Predict Next 240 Data Points
inputs = copiedData[len(copiedData) - len(test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

xTest = []

for i in range(60,inputs.shape[0]):
    xTest.append(inputs[i-60:i,0])

xTest = np.array(xTest)
xTest = np.reshape(xTest, (xTest.shape[0],xTest.shape[1],1))

closingPrice = model.predict(xTest)
closingPrice = scaler.inverse_transform(closingPrice)

rootMeanSquaredError=np.sqrt(np.mean(np.power((test-closingPrice),2)))

# Plot Graph
train = copiedData[:1000]
test = copiedData[1000:]
test['Predictions'] = closingPrice
plt.plot(train['Close'])
plt.plot(test[['Close','Predictions']])
plt.title("LSTM MODEL")